We set torch to only work with CPU

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from dlroms import*
from dlroms.roms import gramschmidt
import dlroms.fespaces as fe

import numpy as np
import matplotlib.pyplot as plt

import torch

import sys
sys.path.insert(1, '../code')

from PODlib import *
from variabilitylib import *

## 0) Data import

In [ ]:
# DATA
dataset = np.load("../data/gaussian_data.npz")
mu, u = dv.tensor(dataset['mu']), dv.tensor(dataset['u'])

ndata, nh = u.shape
p = mu.shape[-1]
print("Trajectories:\t%d." % ndata)
print("FOM dimension:\t%d." % nh)
print("Parameters:\t%d." % p)

# MESH
mesh = fe.loadmesh("../data/gaussian_mesh.xml")
Vh = fe.space(mesh, 'CG', 1)

The training dataset will be 75% of the total trajectories.

In [ ]:
ntrain = int(0.75*ndata)

## 1) Global POD - Ambient space

Compute the space via global POD.

In [ ]:
nA = 30
V, svalues = POD(u[:ntrain], k = nA)
A = gramschmidt(V.unsqueeze(0)).squeeze(0)

Project the data onto the space.

In [ ]:
uproj = project(A, u)

Compute the reconstruction error for the ambient space.

In [ ]:
l2 = L2(Vh)
num2p(mrei(l2)(u[ntrain:], uproj[ntrain:]))

Project the data onto the ambient space, so that we will procede using them.

In [ ]:
uA = projectdown(A, u).squeeze(-1)

print("u shape: ", u.shape)
print("uA shape: ", uA.shape)

## 2) Singular values plots

Define a `weighted_POD` object to compute the singular values.

In [ ]:
n0 = 4 # This value does not really matter
lambda_penalty = 1e-1

w_POD = weighted_POD(A=A,
                     U=torch.t(uA[:ntrain,:]),
                     theta_full=torch.t(mu[:ntrain,:]),
                     n_basis=n0,
                     omega_func=lambda theta, theta_i: omega_weights(theta, theta_i, lambda_penalty=lambda_penalty))   

Compute the singular values.

In [ ]:
s_values_list = []

for theta in mu[:ntrain]:
    _ = w_POD(theta) # To have the singular values, we need a call so that the SVD is done
    s_values_list.append(w_POD.singular_values())

s_values = np.stack(s_values_list)

Look at the related graphs to chose the correct number of basis.

Graph `delta`.

In [ ]:
n_choice_graphs(s_values, which='delta', figsize=(8,4))

Graph `range`.

In [ ]:
n_choice_graphs(s_values, which='range', figsize=(8,4))

Graph `trajectories`.

In [ ]:
n_choice_graphs(s_values, which='trajectories', figsize=(8,4))

Procede with the number of basis $n=4$.

## 3) Compute the scores

Build a definitive `weighted_POD` object with the correct values for the number of basis and use it as the `module` for the `LocalBasis` object that will carry out the computations for the scores.

In [ ]:
n = 4

w_POD = weighted_POD(A=A,
                     U=torch.t(uA[:ntrain,:]),
                     theta_full=torch.t(mu[:ntrain,:]),
                     n_basis=n,
                     omega_func=lambda theta, theta_i: omega_weights(theta, theta_i, lambda_penalty=lambda_penalty))   

In [ ]:
max_mu = mu.max(axis = 0).values
min_mu = mu.min(axis = 0).values

def scaling(theta):
    return theta * (max_mu-min_mu) + min_mu

POD_var = LocalBasis(q=p, 
                     module=w_POD, 
                     p_prime_index_list=np.arange(p), 
                     scaling=scaling)

### 3.1) Derivative based scores

Set a seed for the Monte Carlo estimates.

In [ ]:
seed = 42

In [ ]:
K_0_mean, K_0_var = POD_var.K_j_tot(0, S=100, seed=seed)
print("For j=0 we have:")
print(f"K_sup_j:\t{K_0_mean:.6e} with std:\t{K_0_var:.6e}")

K_1_mean, K_1_var = POD_var.K_j_tot(1, S=100, seed=seed)
print("For j=1 we have:")
print(f"K_sup_j:\t{K_1_mean:.6e} with std:\t{K_1_var:.6e}")

K_2_mean, K_2_var = POD_var.K_j_tot(2, S=100, seed=seed)
print("For j=2 we have:")
print(f"K_sup_j:\t{K_2_mean:.6e} with std:\t{K_2_var:.6e}")

K_3_mean, K_3_var = POD_var.K_j_tot(3, S=100, seed=seed)
print("For j=3 we have:")
print(f"K_sup_j:\t{K_3_mean:.6e} with std:\t{K_3_var:.6e}")

### 3.2) Sensitivity based scores

In [ ]:
sens = POD_var.sensitivity(m = 50, l = 50, seed=seed)
print(f"Sensitivity for j=0:\t{sens[0]:.6e}")
print(f"Sensitivity for j=1:\t{sens[1]:.6e}")
print(f"Sensitivity for j=2:\t{sens[2]:.6e}")
print(f"Sensitivity for j=3:\t{sens[3]:.6e}")